##Setting up environment and importing packages

In [2]:
import os
os.environ["PYSPARK_PYTHON"] = "python3"

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("moive analysis") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()


In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
import math


%matplotlib inline

In [5]:
datapath = "dbfs:/FileStore/tables/covid_train_hydrated.csv"

## reading and cleaning the data

In [7]:
tweets_Df =  spark.read.format("csv").option("header", "true").load(datapath)

In [8]:
#dropping null rows
from pyspark.sql import functions as F

tweets = tweets_Df.select("id","text")

def pre_process(df):
  df_drop = df.filter(df['text'].isNotNull())
  df_drop = df_drop.filter(df_drop['id'].isNotNull())
  df_drop = df_drop.dropDuplicates()
  
  print('After dropping, we have ', str(df_drop.count()), 'row in dataframe')
  return df_drop

tweets_drop = pre_process(tweets)




After dropping, we have 909597 row in dataframe

In [9]:
#dropping text rows that has "false" as value
tweets_drop = tweets_drop.where(~tweets_drop.text.like('false'))
tweets_drop.count()

Out[7]: 901642

In [10]:
#used regular expression matching to get rid of urls
tweets_clean = tweets_drop.select('id', (F.lower(F.regexp_replace('text', "http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", "")).alias('text')))


In [11]:
#total number of tweets left
tweets_clean.count()

Out[36]: 901642

##Building the pre-processing pipeline

In [13]:
#spark nlp does not comes with collection of stop words
#so I used the English stop words from the nltk library
nltk.download('stopwords')

stopwords = nltk.corpus.stopwords.words('english')
stopwords.append("amp")

print ("We use " + str(len(stopwords)) + " stop-words from nltk library.")
print (stopwords[:10])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data] Unzipping corpora/stopwords.zip.
We use 180 stop-words from nltk library.
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [14]:
#building text pre processing pipeline with Spark NLP and Spark ML

import sparknlp

from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *

from pyspark.ml import Pipeline

#document assembler transform raw text into annotator type, common object type used in spark nlp anootator interface

document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")
    
sentence_detector = SentenceDetector() \
    .setInputCols(["document"]) \
    .setOutputCol("sentence") \
    .setUseAbbreviations(True)

#tokenizer transform sentences into individual words for each tweet

tokenizer = Tokenizer() \
    .setInputCols(['sentence']) \
    .setSplitPattern("\p{P}(?<!')") \
    .setMinLength(3)\
    .setMaxLength(16)\
    .setOutputCol("token")

#Removes all dirty characters from text following a regex pattern and transforms words based on a provided dictionary
#this step is probably redundant here since we already did the text cleaning above

normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized")\
    .setLowercase(True)\
    .setCleanupPatterns(["[^\w\d\s]"])

#drop all stop words i.e. words with no inherent meaning in the list of strings
stopwords_cleaner = StopWordsCleaner() \
        .setInputCols(["normalized"]) \
        .setStopWords(stopwords)\
        .setCaseSensitive(False)\
        .setOutputCol("removed")

#retrieving the meaningful part of the word i.e. stem without prefix or suffixes
#stemmer = Stemmer() \
    #.setInputCols(["removed"]) \
    #.setOutputCol("stem")

# using a dictionary or through morphological analysis to return words to their base form
lemmatizer = LemmatizerModel.pretrained() \
     .setInputCols(['removed']) \
     .setOutputCol('lemma')

#generate bi-grams and tri-grams from the lemmatized tokens
ngrams_cum = NGramGenerator() \
            .setInputCols(["lemma"]) \
            .setOutputCol("ngrams") \
            .setN(3) \
            .setEnableCumulative(True)
#produced processed text data
finisher = Finisher() \
    .setInputCols(["removed","lemma","ngrams"]) \
    .setOutputAsArray(True) \
    .setCleanAnnotations(True)

#assembling the pipeline
nlpPipeline = Pipeline(stages=[document_assembler,sentence_detector,tokenizer,normalizer,stopwords_cleaner,lemmatizer,ngrams_cum,finisher])                    

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[ | ][ / ][OK!]

In [15]:
#fitting the model and transforming the data
mod = nlpPipeline.fit(tweets_clean)
cleaned = mod.transform(tweets_clean)

In [16]:
cleaned.select('finished_lemma','finished_ngrams').show(10,truncate = False)

+----------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
finished_lemma |finished_ngrams |
+----------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
[anyone, explain, people, think, azithromycin, treat, coronavirus] |[anyone, explain, people, think, anyone explain, explain people, people think, anyone explain people, explain people think, azithromycin, treat, coronavirus, azithromycin treat, treat coronavirus, azithromycin treat coronavirus] |
[govparsonmo, please, listen, many, voice, across, state, request, plead, take, immediate, preventative, action]|[govparsonmo, please, listen, many, voice, across, state, request, plead, take, immediate, preventative, action, govparsonmo please, please listen, listen many, many voice, voice across, across state, state request, request plead, plead take, take immediate, immediate preventative, preventative action, govparsonmo please listen, please listen many, listen many voice, many voice across, voice across state, across state request, state request plead, request plead take, plead take immediate, take immediate preventative, immediate preventative action]|
[always, look, special, interest, lose, lot, kickback, potus, helm] |[always, look, special, interest, always look, look special, special interest, always look special, look special interest, lose, lot, kickback, potus, helm, lose lot, lot kickback, kickback potus, potus helm, lose lot kickback, lot kickback potus, kickback potus helm] |
[nice, thing, lol] |[nice, thing, lol, nice thing, thing lol, nice thing lol] |
[vibe, need] |[vibe, need, vibe need] |
[around, somebody, call, people, cuz, snitch] |[around, somebody, call, people, cuz, snitch, around somebody, somebody call, call people, people cuz, cuz snitch, around somebody call, somebody call people, call people cuz, people cuz snitch] |
[] |[] |
[terrible, new, rip] |[terrible, new, terrible new, rip] |
[welcome, encourage, insight, present, homelearning, onlineschoolong, sirkenrobinson] |[welcome, encourage, insight, present, homelearning, encourage insight, insight present, present homelearning, encourage insight present, insight present homelearning, onlineschoolong, sirkenrobinson, onlineschoolong sirkenrobinson] |
[peter, frampton, feel] |[peter, frampton, feel, peter frampton, frampton feel, peter frampton feel] |
+----------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [17]:
#using regex matching to unify different namings of COVID-19
training = cleaned.select("id",F.explode("finished_ngrams").alias("elements"))\
          .select("id",F.regexp_replace("elements",'(\w*corona\w*)|(\w*covid\w*)',"coronavirus").alias("elements"))\
          .groupby("id")\
          .agg(F.collect_list("elements").alias("ngrams")) 
        

In [18]:
training.select("ngrams").show(5,truncate = False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
ngrams |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
[marshablackburn, also, coronavirus, hoax, remember, marshablackburn also, also coronavirus, coronavirus hoax, hoax remember, marshablackburn also coronavirus, also coronavirus hoax, coronavirus hoax remember] |
[government, overreact, government overreact, coronavirus, thelouvre, coronavirus thelouvre] |
[confirm, case, coronavirus, south, america, confirm case, case coronavirus, coronavirus south, south america, confirm case coronavirus, case coronavirus south, coronavirus south america, argentina, chile, ecuador, brazil, argentina chile, chile ecuador, ecuador brazil, argentina chile ecuador, chile ecuador brazil, country, temperature, warm, virus, active, spread, country temperature, temperature warm, warm virus, virus active, active spread, country temperature warm, temperature warm virus, warm virus active, virus active spread, 158, confirm, case, 158 confirm, confirm case, 158 confirm case, realdonaldtrump, gop, silent, gop silent, stupid]|
[live, nation, reportedly, suspend, tour, amid, coronavirus, pandemic, live nation, nation reportedly, reportedly suspend, suspend tour, tour amid, amid coronavirus, coronavirus pandemic, live nation reportedly, nation reportedly suspend, reportedly suspend tour, suspend tour amid, tour amid coronavirus, amid coronavirus pandemic] |
[help, italy, help italy, great, trumpland, protect, people, china, help, italy, great trumpland, trumpland protect, protect people, people china, china help, help italy, great trumpland protect, trumpland protect people, protect people china, people china help, china help italy] |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
only showing top 5 rows

In [19]:
from pyspark.ml.feature import CountVectorizer, IDF

#obtaining tf-idf matrix
TF = CountVectorizer(inputCol="ngrams", outputCol="rawFeatures",minDF=0.01,maxDF=0.99,vocabSize = 1000)
tf_model = TF.fit(training)
featurizedData = tf_model.transform(training)
# alternatively, CountVectorizer can also be used to get term frequency vectors

idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

In [20]:
#our bag of words
vocab = tf_model.vocabulary
vocab

Out[14]: ['coronavirus',
 'get',
 'people',
 'trump',
 'test',
 'say',
 'like',
 'need',
 'one',
 'mask',
 'make',
 'time',
 'case',
 'go',
 'know',
 'new',
 'pandemic',
 'day',
 'take',
 'good',
 'see',
 'death',
 'work',
 'think',
 'realdonaldtrump',
 'state',
 'well',
 'would',
 'thank',
 'die',
 'virus',
 'help',
 'home',
 'want',
 'come',
 'back',
 'right',
 'coronavirus coronavirus',
 'still',
 'today',
 'american',
 'life',
 'wear',
 'please',
 'via',
 'let',
 'many',
 'look',
 'great',
 'year',
 'health',
 'week',
 'news',
 'thing',
 'keep',
 'stay',
 'even',
 'school',
 'way',
 'care',
 'give',
 'country',
 'really',
 'number',
 'coronavirus pandemic',
 'tell',
 'open',
 'call',
 'stop',
 'could',
 'first',
 'bad',
 'positive',
 'family',
 'president',
 'use',
 'world',
 'spread',
 'coronavirus case',
 '000',
 'love',
 'report',
 'wear mask',
 'live',
 'much',
 'show',
 'month',
 'start',
 'due',
 'try',
 'dont',
 'america',
 'everyone',
 'every',
 'may',
 'also',
 'read',
 'county',
 'kill',
 'business',
 'happen',
 'safe',
 'lie',
 'close',
 'hope',
 'man',
 'never',
 'job',
 'last',
 'since',
 '2020',
 'put',
 'watch',
 'another',
 'support',
 'feel',
 'update',
 'face',
 'vote',
 'shit',
 'florida',
 'coronavirus test',
 'long',
 'vaccine',
 'high',
 'social',
 'find',
 'public',
 'big',
 'community',
 'real',
 'patient',
 'hospital',
 'talk',
 'friend',
 'house',
 'plan',
 'lose',
 'hear',
 'response',
 'governor',
 'believe']

In [21]:
len(vocab)

Out[48]: 142

## trainning k-means ++ model

In [23]:
#training with all 142 vocabulary
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

evaluator = ClusteringEvaluator()
#spark ml's version of KMeans choose K-means ++ as the initialization algorithm by default
#Optimize choice of k
#for each k, use a different 10% sample to fit and compute silhouette distance
cost = np.zeros(20)
for k in range(2,20):
    kmeans = KMeans().setK(k).setSeed(565).setFeaturesCol("features").setInitSteps(10)
    model = kmeans.fit(rescaledData.sample(False,0.1, seed=42))
    pred = model.transform(rescaledData)
    cost[k] = evaluator.evaluate(pred)

In [24]:
plt.figure(dpi=1200)
fig, ax = plt.subplots(1,1, figsize =(8,6))
ax.plot(range(2,20),cost[2:20], linestyle='--', marker='o')
ax.set_xlabel('k')
ax.set_ylabel('cost')
ax.set_title('Silhouette for k from 2 to 20')
for x,y in zip([6,7,9,12],[cost[6],cost[7],cost[9],cost[12]]):
  xlab = "{:.2f}".format(x)
  plt.annotate(("K = " + xlab),(x,y),textcoords="offset points",xytext = (0,5),ha = 'center')
plt.savefig('this.png')
display()

In [25]:
#fitting the K-means with the optimal K
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

best_K_means = KMeans().setK(7).setSeed(565).setFeaturesCol("features").setInitSteps(10)
best_model = best_K_means.fit(rescaledData)
predictions = best_model.transform(rescaledData)

In [26]:
best_model.summary.clusterSizes

Out[30]: [581559, 116838, 53341, 139137]

In [27]:
centers = best_model.clusterCenters()

In [28]:
def printCenters1(centers):
  for i in range(len(centers)):
    print ("Cluster " + str(i) + " words:", end='')
    order_centroids = centers[i].argsort()[::-1]
    for ind in order_centroids[:10]:
        print(vocab[ind] + ",",end = '')
    print()

In [29]:
len(centers)

Out[92]: 7

In [30]:
printCenters1(centers)

Cluster 0 words:coronavirus,trump,get,say,need,make,realdonaldtrump,well,go,know,
Cluster 1 words:good,like,thank,pandemic,time,coronavirus,via,coronavirus pandemic,coronavirus coronavirus,look,
Cluster 2 words:test,case,coronavirus case,positive,coronavirus test,coronavirus,new,report,death,get,
Cluster 3 words:people,get,coronavirus,mask,wear,trump,say,wear mask,one,die,

In [31]:
def getCenters1(centers):
  centers_summary = {}
  for i in range(len(centers)):
    summary = []
    order_centroids = centers[i].argsort()[::-1]
    for ind in order_centroids[:20]:
       summary.append(vocab[ind])
    centers_summary[i] = summary
  return centers_summary

In [32]:
summary = getCenters1(centers)


Out[97]: {0: ['happen',
 'coronavirus',
 'would',
 'go',
 'people',
 'thing',
 'get',
 'see',
 'know',
 'like',
 'make',
 'think',
 'still',
 'trump',
 'need',
 'say',
 'let',
 'never',
 'time',
 'test'],
 1: ['coronavirus',
 'get',
 'people',
 'trump',
 'test',
 'say',
 'like',
 'need',
 'time',
 'make',
 'know',
 'go',
 'pandemic',
 'take',
 'good',
 'realdonaldtrump',
 'think',
 'work',
 'see',
 'well'],
 2: ['case',
 'coronavirus case',
 'new',
 'coronavirus',
 'death',
 'report',
 'county',
 'state',
 'number',
 'day',
 '000',
 'test',
 'positive',
 'florida',
 'high',
 'today',
 'week',
 'say',
 'people',
 'see'],
 3: ['business',
 'coronavirus',
 'open',
 'help',
 'close',
 'get',
 'support',
 'people',
 'need',
 'work',
 'pandemic',
 'make',
 'state',
 'back',
 'go',
 'take',
 'time',
 'coronavirus pandemic',
 'many',
 'like'],
 4: ['great',
 'coronavirus',
 'job',
 'thank',
 'work',
 'news',
 'time',
 'make',
 'get',
 'see',
 'people',
 'trump',
 'good',
 'america',
 'day',
 'need',
 'like',
 'pandemic',
 'help',
 'one'],
 5: ['one',
 'coronavirus',
 'get',
 'people',
 'love',
 'day',
 'say',
 'thing',
 'time',
 'know',
 'like',
 'test',
 'trump',
 'good',
 'take',
 'see',
 'make',
 'go',
 'need',
 'die'],
 6: ['mask',
 'wear',
 'wear mask',
 'people',
 'coronavirus',
 'social',
 'face',
 'get',
 'please',
 'stay',
 'one',
 'public',
 'spread',
 'say',
 'like',
 'make',
 'see',
 'go',
 'home',
 'everyone']}

In [33]:
def findIdenticalWords(summary):
  out = []
  for i in range(len(summary)):
    xx = set(summary[i])
    out.append(xx)
  u = set.intersection(*out)
  
  return u
  
u = findIdenticalWords(summary)

In [34]:
u

Out[106]: {'coronavirus', 'people'}

using lower dimensionality to see if we have better results?

In [36]:
#limit the dictionary to only top 30 
TF_small = CountVectorizer(inputCol="ngrams", outputCol="rawFeatures",minDF=0.01,maxDF=0.99,vocabSize = 30)
tf_model_small = TF_small.fit(training)
small_features = tf_model_small.transform(training)
# alternatively, CountVectorizer can also be used to get term frequency vectors

idf = IDF(inputCol="rawFeatures", outputCol="features")
smallIdf = idf.fit(small_features)
small_data = smallIdf.transform(small_features)

In [37]:
tf_model_small.vocabulary